<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Solar_panel_dust_detection_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 3.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_datasets as tfds
import zipfile

In [3]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [4]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

In [26]:
def iterative_pruning(model, initial_sparsity, final_sparsity, begin_step, end_step, tdata_loader,vdata_loader, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
          final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer=Adam(.0001, .8, .9),
      loss=binary_crossentropy,
      metrics=['accuracy'])

  callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(tdata_loader, epochs=epochs, validation_data=vdata_loader, callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model

In [27]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

# Load data

In [ ]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)

# Load base model and Evaluate


In [29]:
base_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/final_base_model.h5'
base_model = tf.keras.models.load_model(base_model_path)

In [7]:
base_model_test_acc, base_model_model_size, base_model_executionTime, base_model_evulation_df = evaluate_model_without_saving_stats(base_model, base_model_path, "#0", val_dataset)

9/9 [==============================] - 29s 37ms/step

Model Accuracy: 96.18055820465088 %
Model Size: 648760230.00 bytes
Inference Time is:  3.2716289361317954 s
0.9618055820465088


In [8]:
base_model_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,96.18055820465088 %
3,Loss,11.30877360701561 %
4,Model Size,648760230 bytes
5,Inference Time,3.2716289361317954 sec


# Load iteratively pruned model and Evaluate

In [49]:
ls

drive/                              iteratively_pruned_model_new
fp16_quantized_tflite_model.tflite  iteratively_pruned_stripped_model.h5
iteratively_pruned_model.h5         sample_data/


In [53]:
iteratively_pruned_model_path = "iteratively_pruned_model_new"
iteratively_pruned_model = tf.keras.models.load_model(iteratively_pruned_model_path)

ValueError: ignored

In [39]:
iteratively_pruned_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_sparsity_70.h5'
iteratively_pruned_model_path = "iteratively_pruned_model.h5"

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0, final_sparsity=0.7, begin_step=110, end_step=1010, frequency=100)
}

# pruned_model = prune_low_magnitude(model, **pruning_params)
# iteratively_pruned_model = tf.keras.models.load_model(iteratively_pruned_model_path, custom_objects= )

iteratively_pruned_model = tf.keras.models.load_model(iteratively_pruned_model_path, custom_objects=pruning_params)


# iteratively_pruned_stripped_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_70.h5'
# iteratively_pruned_stripped_model = tf.keras.models.load_model(iteratively_pruned_stripped_model_path)

ValueError: ignored

# Iteratively prun model and Evaluate

In [30]:
# frquency = 100
# target sparsity = 70%
# sparisty for each pruning tep = 70%/100 = 0.07%
# Total Number of Training Samples = 1440
# batch size = 32
# epochs = 10

# possible start and end for the iterative pruning schedule
# Number of learning steps = (1440/32)*10 = 450 (45, 400)
# Number of learning steps = (1440/32)*20 = 900 (90, 800)
# Number of learning steps = (1440/32)*25 = 1125 (110, 1010) -> chosen epoch of 25 epochs
# Number of learning steps = (1440/32)*30 = 1350 (135, 1200)
# Number of learning steps = (1440/32)*35 = 1575 (150, 1400)

# iterative_pruning(
                  #     model, 
                  #     initial_sparsity = 0, 
                  #     final_sparsity   = 0.7, 
                  #     begin_step       = 0, 110  (~10% of 5625) # pruning early on the pruning schedule to get optimal model size
                  #     end_step         = ?, 1010 (~80% of 5625) # pruning only till 80% so that model can still have enough leanring steps to learn and build networks for the data
                  #     train_images, 
                  #     train_labels, 
                  #     epochs           = 3, no change ? since the Number of Steps per Epoch = (Total Number of Training Samples = ?) / (Batch Size = ?) 
                  # )

iteratively_pruned_model, iteratively_pruned_stripped_model = iterative_pruning(base_model, 0, 0.7, 110, 1010, tr_dataset, val_dataset, 25)

Epoch 1/25
36/36 [==============================] - 216s 4s/step - loss: 0.9253 - accuracy: 0.8602 - val_loss: 4.7690 - val_accuracy: 0.6042
Epoch 2/25
36/36 [==============================] - 122s 3s/step - loss: 0.4030 - accuracy: 0.9132 - val_loss: 1.2510 - val_accuracy: 0.8299
Epoch 3/25
36/36 [==============================] - 122s 3s/step - loss: 0.4511 - accuracy: 0.9236 - val_loss: 3.6861 - val_accuracy: 0.6146
Epoch 4/25
36/36 [==============================] - 140s 3s/step - loss: 0.4544 - accuracy: 0.9201 - val_loss: 2.7638 - val_accuracy: 0.6840
Epoch 5/25
36/36 [==============================] - 122s 3s/step - loss: 0.2207 - accuracy: 0.9549 - val_loss: 12.0321 - val_accuracy: 0.5799
Epoch 6/25
36/36 [==============================] - 122s 3s/step - loss: 0.3645 - accuracy: 0.9297 - val_loss: 0.2683 - val_accuracy: 0.9410
Epoch 7/25
36/36 [==============================] - 122s 3s/step - loss: 0.1392 - accuracy: 0.9566 - val_loss: 2.7552 - val_accuracy: 0.6840
Epoch 8/25
3

In [ ]:
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

In [40]:
tf.keras.saving.save_model(
    iteratively_pruned_model, "iteratively_pruned_model_new", overwrite=True, save_format="h5"
)

In [58]:
pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model = iteratively_pruned_model, filepath = "iteratively_pruned_model_new_2_subububu", include_optimizer=False)



# tf.keras.models.save_model(
#     model,
#     filepath,
#     overwrite=True,
#     include_optimizer=True,
#     save_format=None,
#     signatures=None,
#     options=None,
#     save_traces=True,
# )

In [59]:
iteratively_pruned_model_path = "/content/iteratively_pruned_model_new_2.h5"
iteratively_pruned_model = tf.keras.models.load_model(iteratively_pruned_model_path)

ValueError: ignored

In [34]:
iteratively_pruned_model.save("iteratively_pruned_model.h5")
iteratively_pruned_stripped_model.save("iteratively_pruned_stripped_model.h5")

In [35]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(iteratively_pruned_model, "iteratively_pruned_stripped_model.h5", "#1", val_dataset)

9/9 [==============================] - 24s 37ms/step

Model Accuracy: 96.875 %
Model Size: 89656139.00 bytes
Inference Time is:  2.718457433912489 s


In [36]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,96.875 %
3,Loss,13.289713859558105 %
4,Model Size,89656139 bytes
5,Inference Time,2.718457433912489 sec


In [37]:
print_model_weights_sparsity(iteratively_pruned_model)

conv2d_10/kernel:0: 69.23% sparsity  (24126/34848)
conv2d_11/kernel:0: 69.23% sparsity  (425360/614400)
conv2d_12/kernel:0: 69.23% sparsity  (612519/884736)
conv2d_13/kernel:0: 69.23% sparsity  (918778/1327104)
conv2d_14/kernel:0: 69.23% sparsity  (612518/884736)
dense_6/kernel:0: 69.23% sparsity  (26134137/37748736)
dense_7/kernel:0: 69.23% sparsity  (11615172/16777216)
dense_8/kernel:0: 69.24% sparsity  (2836/4096)


In [ ]:
# iteratively_pruned_model.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_saprsity_70.h5')
# iteratively_pruned_stripped_model.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_70.h5')

## Quantization

In [64]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model_2(interpreter, model_path):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in val_dataset:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    startTime = time.time()
    interpreter.invoke()
    executionTime = (time.time() - startTime)/len(val_dataset)

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == val_dataset[1][index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  model_size = get_gzipped_model_size(model_path)
  # Print
  print('\nModel Accuracy:', accuracy*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")
  return accuracy, model_size, executionTime

In [62]:
converter = tf.lite.TFLiteConverter.from_keras_model(iteratively_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_model = converter.convert()

In [63]:
with open('fp16_quantized_tflite_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_model)
interpreter_fp16_quant = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_model.tflite'))
interpreter_fp16_quant.allocate_tensors()

In [65]:
evaluate_model_2(interpreter_fp16_quant, 'fp16_quantized_tflite_model.tflite')

/usr/local/lib/python3.9/dist-packages/numpy/lib/shape_base.py:591: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = asanyarray(a)


ValueError: ignored